In [ ]:
%load_ext autoreload
%autoreload 2

from collections import Counter
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import EoN
import utils
import json
import glob
import seaborn as sns

# Path for data and results

In [ ]:
RES_DIR = "../data/res-sir-sim"
BASE_NET = "{root}/base-network.edgelist".format(root=RES_DIR)
RES_SIM = "{root}/res-sir-simulation.json".format(root=RES_DIR)
t_contact_tracing = 1

# Base network

In [ ]:
G = nx.read_edgelist(BASE_NET)

In [ ]:
avg_deg = EoN.get_PGFPrime(EoN.get_Pk(G))(1)
avg_ex_deg = EoN.get_PGFDPrime(EoN.get_Pk(G))(1) / EoN.get_PGFPrime(EoN.get_Pk(G))(1)

pk = dict(sorted(EoN.get_Pk(G).items()))
qk = dict((k - 1, k * p / avg_deg) for k, p in pk.items())
rk = dict((k - 2, k * (k - 1) * p / (avg_deg * avg_ex_deg)) for k, p in pk.items())
ccdfs = list(map(utils.ccdf_from_pk, [pk, qk, rk]))

# Retrieve the results for simulations 

In [ ]:
resfile_list = glob.glob("{root}/res-sir-simulation-*.json".format(root=RES_DIR))

infected = []
infected_sampled = []
parents = []
to_be_traced_all = []
to_be_traced_25 = []
to_be_traced_10 = []
for resfile in resfile_list:
    with open(resfile, "r") as f:
        result = json.load(f)
    infected += [G.degree(x) for x in result["infected"]]
    infected_sampled += [G.degree(x) for x in result["infected_sampled"]]
    parents += [G.degree(x) for x in result["parents"]]
    to_be_traced = Counter(result["to_be_traced"])
    to_be_traced_all += [G.degree(x) for x in to_be_traced.keys()]
    to_be_traced_25 += [G.degree(x[0]) for x in to_be_traced.most_common(25)]
    to_be_traced_10 += [G.degree(x[0]) for x in to_be_traced.most_common(10)]

infected_ccdf = utils.ccdf_from_data(infected)
parents_ccdf = utils.ccdf_from_data(parents)
traced_ccdf = utils.ccdf_from_data(to_be_traced_all)
traced25_ccdf = utils.ccdf_from_data(to_be_traced_25)
traced10_ccdf = utils.ccdf_from_data(to_be_traced_10)

# Plot results

In [ ]:
cmap = sns.color_palette("Set1").as_hex()

plot_kwargs_for_base_ccdfs = [
    {"label": r"$G_0(x)~ (\sim p_k$)", "color": (0, 0, 0), "linewidth": 0.7},
    {"label": r"$G_1(x)~ (\sim k p_k$)", "color": (0.5, 0.5, 0.5), "linewidth": 0.7},
    {
        "label": r"$G_2(x)~ (\sim k(k-1) p_k)$",
        "color": (0.7, 0.7, 0.7),
        "linewidth": 0.7,
    },
]

sns.set_style("white")
sns.set(font_scale=1)
sns.set_style("ticks")
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(6.49, 4.18), sharey=True)

utils.loglog_plot(
    ccdfs + [infected_ccdf, parents_ccdf],
    kwargs_list=plot_kwargs_for_base_ccdfs
    + [
        {"label": r"Infected ($t={:.1f}$)".format(t_contact_tracing), "color": cmap[1]},
        {
            "label": r"Sampled parents ($t={:.1f}$)".format(t_contact_tracing),
            "color": cmap[0],
        },
    ],
    xlabel=r"$k$",
    ylabel="CCDF",
    ax=ax1,
)

utils.loglog_plot(
    ccdfs + [infected_ccdf, traced_ccdf, traced25_ccdf, traced10_ccdf],
    kwargs_list=[
        {"color": (0, 0, 0), "linewidth": 0.7},
        {"color": (0.5, 0.5, 0.5), "linewidth": 0.7},
        {"color": (0.7, 0.7, 0.7), "linewidth": 0.7},
        {"label": r"Infected".format(t_contact_tracing), "color": cmap[1]},
        {"label": r"Traced, all", "color": "{col}77".format(col=cmap[0]), "ls": ":"},
        {
            "label": r"Traced, top 25",
            "color": "{col}aa".format(col=cmap[0]),
            "ls": "--",
        },
        {"label": r"Traced, top 10", "color": "{col}ff".format(col=cmap[0]), "ls": "-"},
    ],
    xlabel=r"$k$",
    ylabel=None,
    ax=ax2,
)

ax1.annotate(
    "a",
    xy=(0.15, 0.87),
    xycoords="figure fraction",
    horizontalalignment="left",
    verticalalignment="top",
    fontsize=25,
)
ax2.annotate(
    "b",
    xy=(0.6, 0.87),
    xycoords="figure fraction",
    horizontalalignment="left",
    verticalalignment="top",
    fontsize=25,
)


plt.tight_layout()
plt.savefig("fig2-v2.pdf", dpi=300, bbox_inches="tight")